In [1]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import collections

In [2]:
#READ DATA:
IMDB_DF = pd.read_csv("imdb.csv")
imdb_header = list(IMDB_DF.columns.values)

#imdb_header
tbl1 = np.array(imdb_header)
ROTTEN_TOMATOES_DF = pd.read_csv("rotten_tomatoes.csv")
rt_header = list(ROTTEN_TOMATOES_DF.columns.values)
#rt_header
tbl2 = np.array(rt_header)

In [3]:
#creation array to add sort index of matching
# For matching
Pref=np.zeros((len(tbl1),len(tbl2)))
#for Tomatoes
pref_tom=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
ipref_tom=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
#for IMdb
ipref_imdb=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
pref_imdb=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)

In [4]:
#assign the matching ratio between tbl1 and tbl2
for i in range(len(tbl1)):
    for j in range(len(tbl2)):
        Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i],tbl2[j]).ratio())
#df = pd.DataFrame(Pref)
#df.to_csv("file_path.csv")

In [5]:
#assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl2 (17)
for i in range(len(tbl2)):
    ipref_tom[:,i]=np.argsort(Pref[:,i])
    ipref_tom[:,i]=ipref_tom[::-1,i]
    pref_tom[:,i]=np.sort(Pref[:,i])
    pref_tom[:,i]=pref_tom[::-1,i]
#ipref_tom

In [6]:
#assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl1 (13)
for i in range(len(tbl1)):
    #get indexs values of rows and store them in the rows as well
    ipref_imdb[:,i] = np.argsort(Pref[i,:])
    ipref_imdb[:,i] = ipref_imdb[::-1,i]
    pref_imdb[:,i] = np.sort(Pref[i,:])
  #  pref_imdb[:,i] = pref_imdb[::-1,i]
#ipref_imdb

In [7]:
#create list of preferred_rankings_tomatoes with header
preferred_rankings_tomatoes = {}
for i in range(len(tbl2)):
    # for j in range(len(ipref_tom)):
            preferred_rankings_tomatoes[i] = ipref_tom[:,i]
#        pref_Rotten_tomatoes[i][i] = pref_tom[j,i]
#df = pd.DataFrame(preferred_rankings_tomatoes)
#df.to_csv("preferred_rankings_tomatoes.csv")

In [8]:
#create list of preferred_rankings_Imdb with header
preferred_rankings_imdb = {}
for i in range(len(tbl1)):
    # for j in range(len(ipref_tom)):
            preferred_rankings_imdb[i] = ipref_imdb[:,i]
#df = pd.DataFrame(preferred_rankings_imdb)
#df.to_csv("preferred_rankings_imdb.csv")
#preferred_rankings_imdb
#preferred_rankings_imdb[1]

In [45]:
tentative_engagements = []
free_imdbs = []
corresponacy1 = []

In [49]:
#Imdb which still need to propose and get accepted successfully
#Keep track of the headers that "may" end up together
def init_free_imdbs():
    #corresponacy1 = list.clear()
    #'''Initialize the arrays of tomates and imdbs to represent that they're all initially free and not engaged'''
    for imdb in preferred_rankings_imdb.keys():
        free_imdbs.append(imdb)

In [95]:
def begin_matching(imdb):
    #  '''Find the first free Tomaten available to a Imdb at any given time'''
    #print("DEALING WITH %s"%(imdb))
    #print(preferred_rankings_imdb[imdb])
    for tomate in preferred_rankings_imdb[imdb]:
        #Boolean for whether tomate is taken or not
        taken_match = [couple for couple in tentative_engagements if tomate == couple [:][1]]
        if (len(taken_match) == 0):
            #tentatively engage the man and woman
            tentative_engagements.append([imdb, tomate])
            free_imdbs.remove(imdb)
            #print('%s is no longer a free imdb and is now tentatively engaged to %s'%( imdb, tomate))
            break
        elif (len(taken_match) > 0):
            #print('%s is taken already..'%(tomate))
            #print('%s tttt..'%(taken_match[0][0]))
            current_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(taken_match[0][0])
            potential_imdb = list(preferred_rankings_tomatoes[tomate].tolist()).index(imdb)
            #print('potential_imdb %s..'%(current_imdb))
            if (current_imdb < potential_imdb):
                ii=[]
                #print('Tomataen\'s satisfied with %s..'%(taken_match[0][0]))
            else: 
                #print('%s is better than %s'%(imdb, taken_match[0][0]))
                #print('Making %s free again.. and tentatively engaging %s and %s'%(taken_match[0][0], imdb, tomate))
                #The new imdb is engaged
                free_imdbs.remove(imdb)
                #The old imdb is now single
                free_imdbs.append(taken_match[0][0])
                #Update the fiance of the tomaten (tentatively)
                taken_match[0][0] = imdb
                break


In [96]:
def stable_matching():
    '''Matching algorithm until stable match terminates'''
    while (len(free_imdbs) > 0):
        for imdb in free_imdbs:
            begin_matching(imdb)

In [97]:
def output():
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        if(matching > 50.0):
            corresponacy1.append(item)
            print(i ,'. CORRECT \t(',tbl1[item[0]],tbl2[item[1]],')with % of matching',matching)
            i+=1
        else:
                print(i,'. Wrong \t',tbl1[item[0]],tbl2[item[1]],'with % of matching',matching)
                i+=1
    found =len(corresponacy1) - 1
    print('A:Number of the discovered correspondences that are in G(withoutID)',found)
    print('B:Number of all the discovered correspondences',len(corresponacy1))
    print('precision(A/B):',found/len(corresponacy1)*100)
    print('recall(A/Number of all the actual correspondences = 10):',(found/10)*100)
    

In [99]:
def main():
    tentative_engagements.clear()
    free_imdbs.clear()
    corresponacy1.clear()
    init_free_imdbs()
    #print('%s before %s'%tentative_engagements)
    #print(free_imdbs)
    stable_matching()
    output()
main()

0 . CORRECT 	( Id Id )with % of matching 100.0
1 . CORRECT 	( YearRange Year )with % of matching 61.0
2 . CORRECT 	( Director Director )with % of matching 100.0
3 . CORRECT 	( Cast Cast )with % of matching 100.0
4 . CORRECT 	( RatingValue RatingValue )with % of matching 100.0
5 . CORRECT 	( Genre Genre )with % of matching 100.0
6 . CORRECT 	( Description Description )with % of matching 100.0
7 . CORRECT 	( Name Name )with % of matching 100.0
8 . CORRECT 	( Creator Creator )with % of matching 100.0
9 . Wrong 	 ContentRating RatingCount with % of matching 50.0
10 . CORRECT 	( ReleaseDate Release Date )with % of matching 95.0
11 . Wrong 	 Url Actors with % of matching 22.0
12 . CORRECT 	( Duration Duration )with % of matching 100.0
A:Number of the discovered correspondences that are in G(withoutID) 10
B:Number of all the discovered correspondences 11
precision(A/B): 90.9090909090909
recall(A/Number of all the actual correspondences = 10): 100.0
